## first epoc

In [1]:
import pandas as pd
import numpy as np
import pickle
import copy

from algorithm import genetic
from algorithm.kaggle_evaluate import PerplexityCalculator

df_sample = pd.read_csv("sample_submission.csv", index_col = 0)
sample = df_sample.text.to_list()[1]

optimizr = genetic(sample, initial_times = 10000, cross_size = 100, max_stack = 5, crossover_method = "mixture", parent_size = 60, mixture_size = 20, mutation_chances = 2, batch_size = 1024)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [1894.81218161 4303.51538896 3377.86793167 3248.46384335 5440.15319337
 4440.12363163 4953.31585006 2549.74848512 6021.70065959 4405.5703154
 2767.72579685 7977.45725545 2620.4301168  6164.50117552 1947.33831018
 3485.09296991 6460.34176386 3709.86215528 2048.78046502 6665.41502506
 1546.50063078 1577.00262261 1652.68455865 1672.16588125 1705.14639807
 1745.5827796  1745.5827796  1850.9189047  1880.06663623 1880.06663623
 1902.22826681 1917.14762802 1932.18400322 1954.95997683 1970.29291796
 1978.00442641 1978.00442641 2001.32051667 2009.1534637  2009.1534637
 2009.1534637  2009.1534637  2017.01706802 2024.91144962 2040.79302695
 2040.79302695 2048.78046502 2048.78046502 2048.78046502 2048.78046502
 2056.79916504 2072.93084082 2081.04406274 2089.18903891 2097.36589362
 2097.36589362 2113.8157382  2113.8157382  2122.08897908 2130.39460051]


## reputation

In [6]:
parents_indx = copy.deepcopy(optimizr.parents_indx)
genomes = copy.deepcopy(optimizr.genomes)
best_genome = copy.deepcopy(optimizr.best_genome)

In [7]:
optimizr.evaluatr.clear_gpu_memory()
stack = 0
max_stack = 5
evaluatr = None

for _ in range(100) : 
    childs = optimizr.crossover(parents_indx, genomes, dupl = False)
    genomes = optimizr.mutation(childs, mutation_chances = 2)
    
    genome_set = np.unique([" ".join(genome) for genome in genomes] + [best_genome[0]])
    
    if evaluatr != None :
        evaluatr.clear_gpu_memory()
        
    evaluatr = PerplexityCalculator("google/gemma-2-9b", load_in_8bit = False)
    
    print("starting evaluation")
    perplexities = np.array(evaluatr.get_perplexity(genome_set, batch_size = 512))
    
    ## mixture
    per_sum = sum(1/(perplexities**3))
    proba = 1/(perplexities**3)/per_sum
    subset_index = np.random.choice([i for i in range(len(genome_set))], p = proba, size = 20, replace = False)
    sorted_index = perplexities.argsort()
    
    remain_size = 40
    
    for i in range(60) :
        if sorted_index[i] not in subset_index :
            subset_index = np.concat([subset_index, [sorted_index[i]]])
            remain_size -= 1
            
            if remain_size == 0 :
                break
            
    parents_indx = subset_index
    
    if perplexities.min() < best_genome[1] :
        best_genome = [genome_set[perplexities.argmin()], perplexities.min()]
        print(f"best genome : {best_genome}")
        stack = 0
        
    else :
        stack += 1
        
        if stack >= max_stack :
            break
        
    print(f"parents perplexities : {perplexities[parents_indx]}")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

best genome : [np.str_('reindeer jump the scrooge mistletoe and drive laugh elf gingerbread bake walk sleep chimney fireplace ornament family night advent give'), np.float64(902.06583030563)]
parents perplexities : [3173.21325947 2529.90618498 3223.18409717 1745.5827796  1954.95997683
 1932.18400322 1678.71055347 2480.97336985 1947.33831018 1711.82015237
 3051.64936852 2452.06915459 2600.03776785 1522.52436083 3273.94186105
 2097.36589362 2432.98700104 2661.69588665 1843.70285575 1985.74611684
  902.06583031  949.05689557  952.77139924  990.725497   1014.21987511
 1018.18941946 1026.17517805 1026.17517805 1042.33508572 1054.62180187
 1071.22967867 1075.42235309 1079.63143717 1092.35779092 1096.63315843
 1096.63315843 1100.92525926 1109.55992308 1109.55992308 1109.55992308
 1122.63906434 1131.44403165 1135.87237839 1135.87237839 1135.87237839
 1135.87237839 1140.31805718 1140.31805718 1149.26168255 1149.26168255
 1149.26168255 1153.75976559 1153.75976559 1158.27545364 1158.27545364
 115

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [1228.1708774  2276.66913936 1534.46566739 1880.06663623 2452.06915459
 1510.67598225 2934.74251711 4074.48647773 3124.01714781 2946.22877455
 2980.95798704 2276.66913936 4689.70518911 1659.15298313 1725.24612423
 1620.71874567 1464.19736852 1705.14639807 3223.18409717 1633.43020034
  902.06583031 1096.63315843 1113.90261783 1118.26230941 1135.87237839
 1135.87237839 1140.31805718 1140.31805718 1153.75976559 1158.27545364
 1158.27545364 1158.27545364 1162.80881559 1171.92883816 1195.04306336
 1195.04306336 1195.04306336 1199.72032966 1199.72032966 1199.72032966
 1199.72032966 1204.41590227 1204.41590227 1204.41590227 1213.86225327
 1213.86225327 1213.86225327 1213.86225327 1213.86225327 1218.6131758
 1218.6131758  1223.38269292 1223.38269292 1223.38269292 1223.38269292
 1228.1708774  1228.1708774  1228.1708774  1237.80354097 1237.80354097]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [1678.71055347 1932.18400322 7792.65965717 3709.86215528 2189.45131429
 1601.83680035 1364.78201651 1691.87684352 2089.18903891 3539.97519839
 2579.80411341 3949.12770764 1441.49709238 2539.80795783 3918.395352
 2089.18903891 2640.98240493 2395.26703381 2620.4301168  2189.45131429
  902.06583031  934.34313221 1026.17517805 1042.33508572 1062.89330318
 1096.63315843 1096.63315843 1118.26230941 1122.63906434 1122.63906434
 1131.44403165 1135.87237839 1135.87237839 1140.31805718 1153.75976559
 1153.75976559 1153.75976559 1158.27545364 1162.80881559 1162.80881559
 1171.92883816 1176.51563794 1176.51563794 1176.51563794 1176.51563794
 1181.12038996 1181.12038996 1185.74316446 1185.74316446 1185.74316446
 1195.04306336 1195.04306336 1195.04306336 1195.04306336 1199.72032966
 1199.72032966 1199.72032966 1199.72032966 1204.41590227 1213.86225327]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [1939.74635759 2432.98700104 2661.69588665 2206.62339378 1932.18400322
 1978.00442641 3539.97519839 1413.61601476 1985.74611684 1424.70314257
 4011.31741996 1493.07607396 3857.64626283 2232.63437208 2946.22877455
 4726.48700278 2321.57241461 3827.62582144 2312.52146152 4337.26827848
  902.06583031 1092.35779092 1096.63315843 1109.55992308 1118.26230941
 1118.26230941 1131.44403165 1135.87237839 1140.31805718 1140.31805718
 1140.31805718 1140.31805718 1140.31805718 1153.75976559 1153.75976559
 1158.27545364 1162.80881559 1171.92883816 1176.51563794 1181.12038996
 1185.74316446 1199.72032966 1199.72032966 1204.41590227 1213.86225327
 1213.86225327 1213.86225327 1218.6131758  1218.6131758  1218.6131758
 1218.6131758  1223.38269292 1223.38269292 1228.1708774  1228.1708774
 1237.80354097 1237.80354097 1242.64816705 1242.64816705 1242.64816705]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## 개선방안

`-` 상속 구조에 변화

* 부모 유전자를 3개로 증가
* 각 유전자는 문장구조/동사/이외로 나뉘며 해당 구조 하에서 변이가 발생할 수 있음

---

* 동사는 1, 동사가 아니면 0으로 설정

In [1]:
import itertools
import pandas as pd
import numpy as np
import copy
from typing import List
from algorithm.kaggle_evaluate import PerplexityCalculator

In [2]:
def selection(perplexities, parents_size = 20, crossover_method = "mixture", elite_size = 10) :
    perps = np.array(perplexities)
    
    ## select parents index
    if crossover_method == "roulette" :
        per_sum = sum(1/(perps**2))
        proba = 1/(perps**2)/per_sum
        parents_indx = np.random.choice([i for i in range(len(perps))], p = proba, size = parents_size, replace = False)
        
    elif crossover_method == "rank" :
        parents_indx = perps.argsort()[:parents_size]

    elif crossover_method == "mixture" :
        per_sum = sum(1/(perps**2))
        proba = 1/(perps**2)/per_sum
        subset_index = np.random.choice([i for i in range(len(perps))], p = proba, size = parents_size - elite_size, replace = False)
        sorted_index = perps.argsort()
        remain_size = elite_size
        
        for i in range(parents_size) :
            if sorted_index[i] not in subset_index :
                subset_index = np.concat([subset_index, [sorted_index[i]]])
                remain_size -= 1
                
                if remain_size == 0 :
                    break
            
        parents_indx = subset_index
    
    return parents_indx

def mutation_crossover(p, mutation_chances = 1) :
    origin = np.array(p)
    lnth = len(origin)
    
    for _ in range(mutation_chances) :
        if np.random.random() > 0.3 :
            dice = np.random.randint(0, 4) ## choice mutation method
            
            ## swap
            if dice == 0 :
                swap_area = np.random.choice([k for k in range(lnth)], size = 2, replace = False)
                origin[swap_area[0]], origin[swap_area[1]] = origin[swap_area[1]], origin[swap_area[0]]
                
            ## move
            elif dice == 1 :
                moving_indx = np.random.randint(0, lnth)
                mover = origin[moving_indx]
                
                moving_area = np.random.randint(0, lnth)
                
                ## trick
                tmp = list(origin)
                del tmp[moving_indx]
                tmp.insert(moving_area, mover)
                
                origin = np.array(tmp)
                
            ## inverse
            elif dice == 2 :
                if np.random.random() < 0.5 :
                    width = np.random.randint(3, 5)
                    start = np.random.randint(0, lnth-width)
                    origin[start:start+width] = origin[start:start+width][::-1]
                
            ## scramble
            elif dice == 3 :
                if np.random.random() < 0.5 :
                    swap_size = np.random.randint(3, 5)
                    swap_area = np.random.choice([i for i in range(lnth)], size = swap_size, replace = False)
                    
                    origin[swap_area] = np.random.permutation(np.array(origin)[swap_area])
    
    return origin

def crossover(p1, p2, p3, verbs, mutation_chances = 1) :
    structure = [None for _ in range(len(p1))]
    vrbs = [t for t in p2 if t in verbs]
    othrs = [t for t in p3 if t not in verbs]
    
    child = ["" for _ in range(len(p1))]
    
    for i, t in enumerate(p1) :
        structure[i] = t in verbs
    
    if mutation_chances > 0 :
        structure = mutation_crossover(structure, mutation_chances = mutation_chances)
        vrbs = mutation_crossover(vrbs, mutation_chances = mutation_chances)
        othrs = mutation_crossover(othrs, mutation_chances = mutation_chances)

    a = 0
    b = 0
    
    for i, s in enumerate(structure) :
        if s :
            child[i] = vrbs[a]
            a += 1
        else :
            child[i] = othrs[b]
            b += 1
            
    return child

def mutation(genome, mutation_chances = 2) :
    origin = np.array(genome)
    lnth = len(origin)
    
    for _ in range(mutation_chances) :
        ## mutate randomly
        if np.random.random() > 0.5 :
            dice = np.random.randint(0, 4)
            
            ## swap
            if dice == 0 :
                swap_area = np.random.choice([k for k in range(lnth)], size = 2, replace = False)
                origin[swap_area[0]], origin[swap_area[1]] = origin[swap_area[1]], origin[swap_area[0]]
            
            ## move
            elif dice == 1 :
                moving_indx = np.random.randint(0, lnth)
                mover = origin[moving_indx]
                
                moving_area = np.random.randint(0, lnth)
                
                ## trick
                tmp = list(origin)
                del tmp[moving_indx]
                tmp.insert(moving_area, mover)
                
                origin = np.array(tmp)
                
            ## inverse
            elif dice == 2 :
                if np.random.random() < 0.5 :
                    width = np.random.randint(3, 5)
                    start = np.random.randint(0, lnth-width)
                    origin[start:start+width] = origin[start:start+width][::-1]
                
            ## scramble
            elif dice == 3 :
                if np.random.random() < 0.5 :
                    swap_size = np.random.randint(3, 5)
                    swap_area = np.random.choice([i for i in range(lnth)], size = swap_size, replace = False)
                    
                    origin[swap_area] = np.random.permutation(np.array(origin)[swap_area])
    
    return origin

In [3]:
## hyperparameter
initial_times = 1000
batch_size = 1024
rep_times = 100
cross_size = 150
max_stack = 10

## load and shuffle
df_sample = pd.read_csv("sample_submission.csv", index_col = 0)
sample = df_sample.text.to_list()[1]
vrbs = ["walk", "give", "jump", "drive", "bake", "sleep", "laugh"] ## verbs in 1st sample

genomes = ["" for _ in range(initial_times)]

for i in range(initial_times) :
    genomes[i] = np.random.permutation(sample.split())
    
genomes = np.array(genomes)

## evaluation
evaluatr = PerplexityCalculator("google/gemma-2-9b")
perplexities = np.array(evaluatr.get_perplexity([" ".join(genome) for genome in genomes], batch_size = batch_size))

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
## selection
parents_indx = selection(perplexities = perplexities, parents_size = 14, crossover_method = "mixture", elite_size = 8)
best_genome = [" ".join(genomes[perplexities.argmin()]), perplexities.min()]
print(f"best genome : {best_genome[0]}")
print(f"parents perplexities : {perplexities[parents_indx]}")

best genome : reindeer scrooge the night drive advent laugh give family walk sleep mistletoe gingerbread elf fireplace bake jump chimney and ornament
parents perplexities : [7612.14289064 4616.99804929 4371.2858952  8166.63720249 5701.23159625
 6410.0669854  1865.43584159 2001.32051667 2040.79302695 2232.63437208
 2330.65879213 2404.64184391 2414.05334599 2432.98700104]


In [5]:
stack = 0
genome_set = copy.deepcopy(genomes)

## reputation
for i in range(rep_times) :
    pair_parents = list(itertools.permutations(parents_indx, 3))
    childs = []
    
    for pair in pair_parents :
        parents_genome = [genome_set[idx] for idx in pair]
        
        for _ in range(cross_size) :
            crossover_genome = crossover(parents_genome[0], parents_genome[1], parents_genome[2], verbs = vrbs, mutation_chances = 4)
            
            if i <= 10 :
                childs.append(mutation(crossover_genome, mutation_chances = 4))
                
            else :
                childs.append(mutation(crossover_genome, mutation_chances = 2))
                
    genome_set = np.unique([" ".join(genome) for genome in genome_set] + [best_genome[0]]) ## elitism

    evaluatr.clear_gpu_memory()
    evaluatr = PerplexityCalculator("google/gemma-2-9b")
    
    perplexities = np.array(evaluatr.get_perplexity(genome_set, batch_size = batch_size))
    
    parents_indx = selection(perplexities = perplexities, parents_size = 14, crossover_method = "mixture", elite_size = 8)
    
    if perplexities.min() < best_genome[1] :
        best_genome = [genome_set[perplexities.argmin()], perplexities.min()]
        print(f"best genome : {best_genome}")
        stack = 0
        
    else :
        stack += 1
        
        if stack >= max_stack :
            break
        
    genome_set = [genome.split() for genome in genome_set] ## error solved
    
    print(f"parents perplexities : {perplexities[parents_indx]}")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [2789.43333932 2767.72579685 7207.03203573 5791.01293001 2001.32051667
 6021.70065959 1865.43584159 2040.79302695 2232.63437208 2330.65879213
 2404.64184391 2414.05334599 2432.98700104 2490.68362507]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [8969.29628578 2714.19313404 9774.18494704 2724.81618561 5314.13218038
 1865.43584159 2001.32051667 2040.79302695 2232.63437208 2330.65879213
 2404.64184391 2414.05334599 2432.98700104 2490.68362507]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

...

parents perplexities : [ 5150.63352163  2569.74641034  2569.74641034  5397.81758535
 17288.77109334  5314.13218038  1865.43584159  2001.32051667
  2040.79302695  2232.63437208  2330.65879213  2404.64184391
  2414.05334599  2432.98700104]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [2714.19313404 4203.82456235 4203.82456235 5314.13218038 4371.2858952
 8761.52276111 1865.43584159 2001.32051667 2040.79302695 2232.63437208
 2330.65879213 2404.64184391 2414.05334599 2432.98700104]


KeyboardInterrupt: 

: 

> 무언가 하자가 있음에 분명함. 애초에 퍼플렉시티 계산이 왜이렇게 빠른 거임? 중복이 엄청 많은건가? 아니면 제대로 생성이 안되나? 확인 요구됨